In [1]:
import os
import argparse

from lsst.ts.wep.Utility import FilterType
from lsst.ts.ofc.Utility import InstName
from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory

from lsst.ts.phosim.telescope.TeleFacade import TeleFacade
from lsst.ts.phosim.PhosimCmpt import PhosimCmpt
from lsst.ts.phosim.Utility import getPhoSimPath, getAoclcOutputPath
from lsst.ts.phosim.PlotUtil import plotFwhmOfIters


In [3]:
%%time


#generic default setups 
numPro = 1
iterNum = 5 
rotCamInDeg=0.0

# Survey parameters
filterType = FilterType.REF

# Prepare the components 

#############################
#### _preparePhosimCmpt #####
#############################

#phosimCmpt = _preparePhosimCmpt(phosimDir, filterType, 0.0, numPro)
# part of 
#_preparePhosimCmpt(phosimDir, filterType, rotAngInDeg, numPro)
rotAngInDeg = 0 # default 
numPro = 1 # default 
# Set the Telescope facade class
tele = TeleFacade()
tele.addSubSys(addCam=True, addM1M3=True, addM2=True)

# set phosim path variable name, by default its phosimPathVar = "PHOSIMPATH"
# so it returns os.environ["PHOSIMPATH"], 
# i.e. points to dir set by setup_aos.bash
# '/astro/store/epyc/projects/lsst_comm/phosim_syseng4/'
phosimDir = getPhoSimPath()
tele.setPhoSimDir(phosimDir)


CPU times: user 6min 16s, sys: 5.55 s, total: 6min 21s
Wall time: 6min 23s


amazing that the cell above takes so long to work out ! 

In [5]:
%%time

numPro = 1
#iterNum = 5 
#rotCamInDeg=0.0

# Prepare the phosim component
phosimCmpt = PhosimCmpt(tele)

# Set the telescope survey parameters
boresight = (0, 0)
zAngleInDeg = 27.0912
phosimCmpt.setSurveyParam(filterType=filterType, boresight=boresight,
                          zAngleInDeg=zAngleInDeg, rotAngInDeg=rotAngInDeg)

# Update the number of processor if necessary

if (numPro > 1):
    settingFile = phosimCmpt.getSettingFile()
    settingFile.updateSetting("numPro", numPro)

# Set the seed number for M1M3 surface
seedNum = 6
phosimCmpt.setSeedNum(seedNum)



CPU times: user 4.05 ms, sys: 2 µs, total: 4.05 ms
Wall time: 3.84 ms


In [6]:
%%time

#############################
##### _prepareOfcCalc #####
#############################

# prepare OFC calculation
ofcCalc = OFCCalculationFactory.getCalculator(InstName.COMCAM)
ofcCalc.setFilter(filterType)
ofcCalc.setRotAng(rotAngInDeg)
ofcCalc.setGainByPSSN()

CPU times: user 16.6 s, sys: 556 ms, total: 17.1 s
Wall time: 17.7 s


In [7]:
%%time
# Set the telescope state to be the same as the OFC
state0 = ofcCalc.getStateAggregated()
phosimCmpt.setDofInUm(state0)

CPU times: user 1.27 ms, sys: 9 µs, total: 1.28 ms
Wall time: 1.01 ms


check how long one iteration takes 

In [10]:
%%time

# Do the iteration
obsId = 9006000
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
baseOutputDir = ''

iterNum = 1
rotCamInDeg=0.0


#for iterCount in range(iterNum):

# Set the observation Id
phosimCmpt.setSurveyParam(obsId=obsId)

# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

# Set the output directory
outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
phosimCmpt.setOutputDir(outputDir)

# Set the output image directory
outputImgDir = os.path.join(baseOutputDir, iterDirName,
                                outputImgDirName)
phosimCmpt.setOutputImgDir(outputImgDir)

# Generate the OPD image
argString = phosimCmpt.getComCamOpdArgsAndFilesForPhoSim()
phosimCmpt.runPhoSim(argString)

# Analyze the OPD data
# Rotate OPD in the reversed direction of camera
phosimCmpt.analyzeComCamOpdData(zkFileName=opdZkFileName,
                                rotOpdInDeg=-rotCamInDeg,
                                pssnFileName=opdPssnFileName)

# Get the PSSN from file
pssn = phosimCmpt.getOpdPssnFromFile(opdPssnFileName)
print("Calculated PSSN is %s." % pssn)

# Get the GQ effective FWHM from file
gqEffFwhm = phosimCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
print("GQ effective FWHM is %.4f." % gqEffFwhm)

    

Calculated PSSN is [0.59515269 0.59006253 0.59086933 0.59333341 0.58535632 0.58658711
 0.59610244 0.58819379 0.58696264].
GQ effective FWHM is 0.5429.


NameError: name '_getComCamSensorNameList' is not defined

In [12]:
argString

'/data/epyc/projects/lsst_comm/iter0/pert/opd.inst -i lsst -e 1 -c /data/epyc/projects/lsst_comm/iter0/pert/opd.cmd -o /data/epyc/projects/lsst_comm/iter0/img > /data/epyc/projects/lsst_comm/iter0/img/opdPhoSim.log 2>&1'

In [11]:
###################################
#### _getComCamSensorNameList #####
###################################

# Set the FWHM data
#refSensorNameList = _getComCamSensorNameList()
refSensorNameList = ["R22_S00", "R22_S01", "R22_S02", "R22_S10", "R22_S11",
                      "R22_S12", "R22_S20", "R22_S21", "R22_S22"]
###################################

listOfFWHMSensorData = phosimCmpt.getListOfFwhmSensorData(
    opdPssnFileName, refSensorNameList)
ofcCalc.setFWHMSensorDataOfCam(listOfFWHMSensorData)

# Simulate to get the wavefront sensor data from WEP and calculate
# the DOF
listOfWfErr = phosimCmpt.mapOpdDataToListOfWfErr(
    opdZkFileName, refSensorNameList)
ofcCalc.calculateCorrections(listOfWfErr)

# Set the new aggregated DOF to phosimCmpt
dofInUm = ofcCalc.getStateAggregated()
phosimCmpt.setDofInUm(dofInUm)

# Save the DOF file
phosimCmpt.saveDofInUmFileForNextIter(
    dofInUm, dofInUmFileName=dofInUmFileName)

# Add the observation ID by 1
obsId += 1
